## About This Notebook

In [34]:
%pip install lightning
%pip install wandb
%pip install pandas numpy matplotlib wandb torch lightning transformers tqdm scikit-learn
%pip install transformers==3.2.0
%pip install transformers[sentencepiece]
%pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages

In [35]:
import pandas as pd
import numpy as np
import pprint
import json
import xml.etree.ElementTree as ET
import re
import random
import os

import matplotlib.pyplot as plt
import numpy as np
import wandb
import torch
import lightning as L
import transformers
import random

from tqdm import tqdm
from collections import Counter
from tqdm.notebook import tqdm
from typing import *

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.dummy import DummyClassifier

from torch import optim, nn, utils, Tensor
import torch.nn.functional as F

from lightning.pytorch.loggers import WandbLogger

## Data Processing

In [26]:
train = pd.read_json("/mnt/c/Users/hakve/OneDrive - uniroma1.it/Desktop/HW1_B_205432/data/SardiStance-train.jsonl", lines=True)
test = pd.read_json("/mnt/c/Users/hakve/OneDrive - uniroma1.it/Desktop/HW1_B_205432/data/SardiStance-test.jsonl", lines=True)
train

,text,choices,label
0,CONTRORDINE COMPAGNI. \nFATE GIRARE LA VOCE CH...,"[AGAINST, NONE, FAVOR]",0
1,A proposito dell'incontro delle @6000sardine c...,"[NONE, AGAINST, FAVOR]",0
2,Care sardine che richiamate in continuazione i...,"[AGAINST, NONE, FAVOR]",0
3,"Domanda ad una sardina "" secondo te Qual è il ...","[AGAINST, NONE, FAVOR]",0
4,A salvini non gli frega nulla delle sardine no...,"[NONE, AGAINST, FAVOR]",0
...,...,...,...
2127,"Pidini non montatevi la testa, non avete perso...","[NONE, AGAINST, FAVOR]",0
2128,Volete sapere perché il pd nonostante tutto di...,"[NONE, AGAINST, FAVOR]",0
2129,#Omnibusla7 Telese in piena fregola per il cap...,"[NONE, AGAINST, FAVOR]",0
2130,Noto con piacere che le librerie si stanno rip...,"[NONE, AGAINST, FAVOR]",0


In [27]:
## Encoding target
target_map = {
    "AGAINST": 0,
    "NONE" : 1,
    "FAVOR" : 2
}

## Mapping function
def map_label_to_target(label, choices):
  return target_map[choices[label]]

## Creating target column
train['target'] = train.apply(lambda row: map_label_to_target(row['label'], row['choices']), axis=1)
test['target'] = test.apply(lambda row: map_label_to_target(row['label'], row['choices']), axis=1)
test

,text,choices,label,target
0,Quando vedo sardine pieddini e pidiessini mi c...,"[AGAINST, NONE, FAVOR]",0,0
1,Sardine pidini sinistri di vario tipo cinque s...,"[AGAINST, NONE, FAVOR]",0,0
2,#Salvinivergognati ha scatenato la macchina de...,"[FAVOR, NONE, AGAINST]",0,2
3,Oggi a San Giovanni con le sardine ce stato un...,"[AGAINST, NONE, FAVOR]",0,0
4,"È sabato tutti a #reggioemilia ore 18,30 piazz...","[FAVOR, NONE, AGAINST]",0,2
...,...,...,...,...
1105,Sono nella #menaide del Pd. #Sardine. Ps: mena...,"[AGAINST, NONE, FAVOR]",0,0
1106,NE CARNE NE PESCE !\nSARDINE O MORTADELLA ? \n...,"[AGAINST, NONE, FAVOR]",0,0
1107,#iostoconsalvini ottimo hastag per bloccare #S...,"[AGAINST, NONE, FAVOR]",0,0
1108,"La ""sardina"" Jasmine scrive a Salvini: ""Combat...","[FAVOR, NONE, AGAINST]",0,2


In [28]:
## Custom utils.data.Dataset and LightningDataLoader
class StanceDataset(utils.data.Dataset):
  def __init__(self, data: pd.DataFrame, tokenizer: transformers.PreTrainedTokenizer, max_length, remove_prompt=False, extra_verbose=False):
    self.data = data
    self.total_len = len(self.data)
    self.max_length = max_length
    self.tokenizer = tokenizer
    self.remove_prompt = remove_prompt
    self.extra_verbose = extra_verbose

  def __len__(self):
    return self.total_len

  def __getitem__(self, index):
    sample = self.data.iloc[index]
    text = sample["text"]
    target = sample["target"]

    encoding = self.tokenizer(
        text,
        max_length=self.max_length,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt",
    )

    input_ids = encoding["input_ids"].squeeze()
    attention_mask = encoding["attention_mask"].squeeze()

    return input_ids, attention_mask, target

In [29]:
class StanceDataModule(L.LightningDataModule):
    def __init__(self, train_df: pd.DataFrame, test_df: pd.DataFrame, max_length, tokenizer: transformers.PreTrainedTokenizer, batch_size = 128, num_workers=0, remove_prompt=False, extra_verbose=False):
        super().__init__()
        self.max_length = max_length
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.tokenizer = tokenizer
        self.extra_verbose = extra_verbose

        self.train_df = train_df
        self.test_df = test_df

    def prepare_data(self):
        pass

    def setup(self, stage=None):
      self.train_dataset = StanceDataset(
            self.train_df,
            self.tokenizer,
            self.max_length,
            self.extra_verbose
        )

      self.test_dataset = StanceDataset(
            self.test_df,
            self.tokenizer,
            self.max_length,
            self.extra_verbose
        )

    def train_dataloader(self):
        return utils.data.DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
        )

    def test_dataloader(self):
        return utils.data.DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
        )

    def val_dataloader(self):
        return utils.data.DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
        )

## Hyperparameters

In [30]:
## [!] Feature number to LSTM must be equal to max token lenght
p = dict(
    n_features = 16,
    hidden_size = 64,
    batch_size = 32,
    num_layers = 2,
    dropout = 0.5,
    max_length = 16,
    learning_rate = 0.001,
    criterion = nn.CrossEntropyLoss(),
    max_epochs = 10,
    num_workers = 4,
    num_classes = 3,
    vocab_size = 32000,
    embedding_dim = 200
)

## Main Model
**Prepare your dataset:** You need to have your dataset prepared with input features (text data) and corresponding labels (class labels).

**Preprocess the text data:** Tokenize your text data and convert it into numerical representations. You can use techniques like tokenization, padding, and possibly filtering out stopwords.

In [31]:
class LSTMClassifier(L.LightningModule):
    def __init__(self, n_features, hidden_size, vocab_size, num_classes, num_layers, dropout, learning_rate, criterion):
        super(LSTMClassifier, self).__init__()
        self.n_features = n_features
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.criterion = criterion
        self.learning_rate = learning_rate
        self.num_classes = num_classes

        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.lstm = nn.LSTM(input_size=hidden_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            dropout=dropout,
                            batch_first=True)
        self.linear = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        lstm_out = lstm_out[:, -1, :]
        logits = self.linear(lstm_out)
        return logits

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, target = batch
        logits = self(input_ids)
        loss = self.criterion(logits, target)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, targets = batch
        outputs = self(input_ids)
        loss = self.criterion(outputs, targets)
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids, attention_mask, targets = batch
        outputs = self(input_ids)
        loss = self.criterion(outputs, targets)
        self.log("test_loss", loss)
        return loss

In [ ]:
# Train the model (hint: here are some helpful Trainer arguments for rapid idea iteration)
# L.seed_everything(1)
tokenizer = transformers.T5Tokenizer.from_pretrained("t5-base")
## wandb_logger = WandbLogger(log_model="all")

main_trainer = L.Trainer(
    max_epochs=p['max_epochs'],
##    log_every_n_steps=1,
##    logger=wandb_logger,
    accelerator="auto",
)

model = LSTMClassifier(
    n_features = p['n_features'],
    hidden_size = p['hidden_size'],
    criterion = p['criterion'],
    num_layers = p['num_layers'],
    dropout = p['dropout'],
    learning_rate = p['learning_rate'],
    num_classes = p['num_classes'],
    vocab_size = p['vocab_size']
)

dm = StanceDataModule(
    train,
    test,
    max_length=p["max_length"],
    tokenizer=tokenizer,
    batch_size=p["batch_size"],
    num_workers=p["num_workers"]
)

## wandb_logger.watch(model, log="all")

print("Parameters")
pprint.pp(p)
main_trainer.fit(model, dm)
main_trainer.test(model, datamodule=dm)